In [26]:
import numpy as np
import glob
import cv2
import os
import shutil

import torch

import matplotlib.pyplot as plt

# Important so PyTorch can load LSTM Model Weights
from Utils.PTModel.Models import VGLCLSTMModel
from Utils.Data.DataLoading import TextTileToImage

MODELNAME = "VGCLLSTM"
NUMBEROFLEVELS = 150
RANDOMSEED = 10

In [8]:
model = torch.load(f"Models/{MODELNAME}/LodeRunnerLSTM.pt")

/tmp/ipykernel_1037/2861288654.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f"Models/{MODELNAME}/LodeRunnerLSTM.pt")


In [30]:
# Deletes the directory of the lode runner levels folder
shutil.rmtree(f"Models/{MODELNAME}/LodeRunnerLevels")

In [31]:
# To safeguard against accidentally deleting the wrong model folder
if os.path.isdir(f"Models/{MODELNAME}/LodeRunnerLevels"):
    raise RuntimeError(f"Models/{MODELNAME}/LodeRunnerLevels path already exists. Please delete it an re-run the code.")

os.mkdir(f"Models/{MODELNAME}/LodeRunnerLevels")

device = "cuda" if torch.cuda.is_available() else "cpu"

lrStringLevelPaths = sorted(glob.glob("../data/vglc/Lode Runner/Processed/Level*.txt"))
lrStringNames = [path[path.rfind("/")+1:path.rfind(".txt")] for path in lrStringLevelPaths]

rowLength = 32
numOfRows = 22

columnRefArray = np.array([np.arange(0, 32) for i in range(numOfRows+5)]).flatten()

tiles = {
    "B" : 0,
    "b" : 1,
    "@" : 2,
    "." : 3,
    "-" : 4,
    "#" : 5,
    "G" : 6,
    "E" : 7,
    "M" : 8,
}

model.to(device)
model.eval()

padSize = rowLength * 3 # lode runner row length * 3 as paper uses previous 3 rows for lstm
N = padSize

tilesIdx = [ "B", "b", "@", ".", "-", "#", "G", "E", "M" ]

np.random.seed(RANDOMSEED)
randStartPoint = np.random.randint(0, len(lrStringLevelPaths)-NUMBEROFLEVELS) if len(lrStringLevelPaths)-NUMBEROFLEVELS > 0 else 0
print(f"Random Start Point: {randStartPoint}")

for i, levelStringPath in enumerate(lrStringLevelPaths[randStartPoint:randStartPoint+NUMBEROFLEVELS]):

    input = []
    column = []
    target = []

    with open(levelStringPath, "r") as f:
        levelString = np.array([list(line[:-1]) for line in f.readlines()]).flatten()
        levelString = np.array([tiles[tileChar] for tileChar in levelString])

    for j in range(len(levelString) - N):

        padLength = (N - j) if j < N else 0
        RowCutOff = 0 if j <= N else RowCutOff+1

        dataI = np.concatenate((np.zeros(shape=(padLength)), levelString[RowCutOff:j]), axis=0)

        dataT = levelString[j:j+N]

        levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[RowCutOff:j]), axis=0)
        dataC = np.zeros(shape=(N, 256))
        for t in range(N): dataC[t][int(levelIdx[t])] = 1

        column.append(dataC)
        input.append(dataI)
        target.append(dataT)

    input = np.array(input)
    target = np.array(target)
    column = np.array(column)

    # print(f"Input shape: {input.shape}")
    # print(f"target shape: {target.shape}")
    # print(f"column shape: {column.shape}")

    input = input.reshape(input.shape[0], input.shape[1], 1)
    target = target.reshape(target.shape[0], target.shape[1], 1)

    # print(f"Input shape: {input.shape}")
    # print(f"target shape: {target.shape}")
    # print(f"column shape: {column.shape}")

    levelImage = np.empty(shape=((22), 32), dtype=str)

    for b in range(8):
        
        if b == 0:
            inputTensor = torch.tensor(input[b*N], dtype=torch.float32).to(device)
        else:
            inputTensor = torch.tensor(previousCol.reshape(previousCol.shape[0], 1), dtype=torch.float32).to(device)
        
        # print(f"input size: {inputTensor.size()}")

        if b == 7:
            targetTensor = torch.tensor(target[-33], dtype=torch.float32).to(device)
            columnTensor = torch.tensor(column[-33], dtype=torch.float32).to(device)
        else:
            targetTensor = torch.tensor(target[b*N], dtype=torch.float32).to(device)
            columnTensor = torch.tensor(column[b*N], dtype=torch.float32).to(device)
        
        # print(f"target size: {targetTensor.size()}")
        # print(f"column size: {columnTensor.size()}")

        # print("Running Model")

        nextCol = model(inputTensor, targetTensor, columnTensor)
        nextCol = nextCol.cpu().detach().numpy()
        # print(f"next col size: {nextCol.shape}")
        nextCol = np.argmax(nextCol, axis=1)
        # print(f"next col size: {nextCol.shape}")

        previousCol = nextCol
        nextCol = nextCol.reshape(3, 32)

        if b == 7:
            nextCol = nextCol[0, :].reshape(1, 32)

        partLevelImage = np.zeros(shape=(nextCol.shape[0], nextCol.shape[1]), dtype=str)

        for t, row in enumerate(nextCol):
            for j, embedding in enumerate(row):
                print(embedding)
                partLevelImage[t, j] = tilesIdx[int(embedding)]
        
        # print(f"Part Level Image Shape: {partLevelImage.shape}")
        levelImage[b*3:(b+1)*3, :] = partLevelImage

        if b == 7:
            levelImage[b*3:(b*3)+1, :] = partLevelImage
        else:
            levelImage[b*3:(b+1)*3, :] = partLevelImage

    levelImage = TextTileToImage(levelImage, 16, "../data/tomData/sprites/loderunner")

    # plt.figure()
    # plt.imshow(levelImage)

    cv2.imwrite(f"Models/{MODELNAME}/LodeRunnerLevels/{lrStringNames[i]}.png", cv2.cvtColor(levelImage, cv2.COLOR_BGR2RGB))

Random Start Point: 0
Input shape: (608, 96)
target shape: (608, 96)
column shape: (608, 96, 256)
Input shape: (608, 96, 1)
target shape: (608, 96, 1)
column shape: (608, 96, 256)
input size: torch.Size([96, 1])
target size: torch.Size([96, 1])
column size: torch.Size([96, 256])
Running Model
next col size: (96, 9)
next col size: (96,)
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
1
1
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
Part Level Image Shape: (3, 32)
input size: torch.Size([96, 1])
target size: torch.Size([96, 1])
column size: torch.Size([96, 256])
Running Model
next col size: (96, 9)
next col size: (96,)
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
1
1
1
1
4
4
4
4
4
4
4
4
4
4
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Part Level Image Shape: (3, 32)
input size: torch.Size([96, 1])
target size: torch.Size(